In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack, unique
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord, find_near_objid, find_near_ra_dec, fill_values_by_query
from easyquery import Query, QueryMaker

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import *
print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])

    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

base = annotate_base(base)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))

def add_points(*queries, **plot_options):
    t = Query(*queries).filter(base, ["r_mag", "sb_r", "gr"])
    plot_options.setdefault("rasterized", True)
    plot_options.setdefault("lw", 0)
    ax[1].scatter(t["r_mag"], t["gr"], **plot_options)
    if "label" in plot_options:
        del plot_options["label"]
    ax[0].scatter(t["r_mag"], t["sb_r"], **plot_options)    

is_p1_sats = Query(C.is_sat, saga.host_catalog.construct_host_query("paper1_complete"), QueryMaker.in1d("OBJID", [4443380000000000475, 3115540000000003218], invert=True))
    
add_points(~C.has_spec, c="grey", s=0.05, label="Galaxies, w/o $z$")
add_points(C.has_spec, ~C.is_low_z, c="C0", s=0.1, label="Galaxies, $z>0.02$")
add_points(C.has_spec, C.is_low_z, ~C.is_sat, c="C1", marker='o', s=5, alpha=1, label="Galaxies $z<0.02$")

add_points(is_p1_sats, c="C2", marker='^', s=14, label="Sat. (Paper I)", lw=1, zorder=99)
add_points(Query(C.is_sat, ~is_p1_sats), c="C2", marker='s', s=10, label="Sat. (This work)", lw=1, zorder=99)
add_points(Query("human_selected >= 2", C.has_been_targeted, ~C.has_spec),  edgecolors="C5", c="None", marker='o', s=10, alpha=0.8, lw=1, label="dLSBGs, w/o $z$")

r = np.linspace(11.8, 21, 91)
ax[0].plot(r, 0.7*(r-14) + 18.5, color='C4', ls="--", alpha=1, label='Primary targeting region')
ax[1].plot(r, -0.06*(r-14) + 0.9, color='C4', ls="--", alpha=1)

ax[0].set_xlim(11.8, 21)
ax[1].set_xlim(11.8, 21)

ax[0].set_ylim(18, 26.5)
ax[1].set_ylim(-0.05, 1.55)

ax[1].set_xlabel(r'$r_o$')
ax[1].set_ylabel(r'$(g-r)_o$')
ax[0].set_xlabel(r'$r_o$')
ax[0].set_ylabel(r'${\mu}_{r_o,{\rm eff}}$')

lgnd = ax[0].legend(loc="lower right", scatterpoints=3, frameon=True, fontsize=12.5, handletextpad=0.5)
#lgnd.legendHandles[0]._sizes = [30]
#lgnd.legendHandles[1]._sizes = [20]

lgnd = ax[1].legend(loc="upper left", scatterpoints=3, frameon=True, fontsize=12.5, ncol=2, handletextpad=0.5, columnspacing=1)
lgnd.legendHandles[0]._sizes = [10]
lgnd.legendHandles[1]._sizes = [10]
lgnd.legendHandles[2]._sizes = [15]
lgnd.legendHandles[3]._sizes = [15]

fig.tight_layout()
#fig.colorbar(c, cax=fig.add_axes((0.90, 0.52, 0.01, 0.4)), label=r'$p_{\rm GMM}$')
plt.savefig('/home/yymao/Downloads/targeting-cuts.pdf', dpi=200, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))

def add_points(*queries, **plot_options):
    t = Query(*queries).filter(base, ["radius_", "gr", "Mr_", "sb_r"])
    plot_options.setdefault("rasterized", True)
    plot_options.setdefault("lw", 0)
    ax[0].scatter(t["Mr_"], t["radius_"], **plot_options)
    ax[1].scatter(t["sb_r"], t["gr"], **plot_options)

has_poor_z = Query("ZQUALITY > 1", "ZQUALITY < 3")
    
add_points(~C.has_spec, c="grey", s=0.05)
add_points(C.has_spec, c="C0", s=0.1)
add_points(Query(C.is_sat), c="C2", marker='s', s=15, label="Confirmed sats")
#add_points("human_selected", has_poor_z, C.is_low_z, c="C3", marker='o', s=15, alpha=1, label="LSBG, poor $z<0.02$")
#add_points("human_selected", C.gr_cut, "ZQUALITY == -1", c="C8", marker='x', s=15, alpha=1, lw=1, label="LSBG, not targeted")
add_points("human_selected > 0", C.gr_cut, "ZQUALITY > -1", "ZQUALITY <= 2", c="C5", marker='x', s=15, alpha=1, lw=1, label="LSBG, failed $z$")
#add_points("ZQUALITY >= ", C.is_very_low_z, c="C5", marker='x', s=15, alpha=1, lw=1,)
r = np.linspace(-21, -11, 10)
ax[0].plot(r, 10**(-0.1*(r+18)), color='C4', ls="--", alpha=0.9)
#ax[0].plot(r, 0.7*(r-14) + 18.7, color='C4', ls="--", alpha=0.4)
##ax[0].plot(r, 0.6*(r-14) + 20.4, color='C8', ls="--", alpha=0.9)
##ax[1].plot(r, -0.06*(r-14) + 1.1, color='C8', ls="--", alpha=0.8, label='Targeting cuts')
#ax[1].plot(r, -0.06*(r-14) + 0.9, color='C4', ls="--", alpha=0.9)
#ax[1].plot(r, -0.06*(r-14) + 1.1, color='C4', ls="--", alpha=0.4)
#ax[1].axhline(0.85, c='C6', ls="--", alpha=0.5)

ax[0].set_xlim(-21, -11)
ax[1].set_xlim(18, 27)

ax[0].set_yscale("log")
ax[0].set_ylim(0.1, 10)
ax[1].set_ylim(-0.05, 1.6)

ax[1].set_xlabel(r'$M_{r,o}$')
ax[1].set_ylabel(r'$(g-r)_o$')
ax[0].set_xlabel(r'$M_{r,o}$')
ax[0].set_ylabel(r'$R_{\mathrm{eff},r}$ [kpc]')

#lgnd = ax[0].legend(loc="lower right", scatterpoints=3, frameon=True, fontsize=12)
#lgnd.legendHandles[0]._sizes = [30]
#lgnd.legendHandles[1]._sizes = [20]

lgnd = ax[1].legend(loc="upper left", scatterpoints=2, frameon=True, fontsize=10, ncol=2)
#lgnd.legendHandles[2]._sizes = [10]
#lgnd.legendHandles[3]._sizes = [10]
#lgnd.legendHandles[4]._sizes = [10]

fig.tight_layout()

#fig.colorbar(c, cax=fig.add_axes((0.90, 0.52, 0.01, 0.4)), label=r'$p_{\rm GMM}$')
#plt.savefig('/home/yymao/Downloads/targeting-cuts.pdf', dpi=200)

In [ ]:
Query(C.lsbg_cut, C.gr_cut, "Mr_ >= -15.4", "Mr_ < -12", ~C.has_spec).count(base)

In [ ]:
sid = np.array([4416240000000002817, 4456920000000002940, 4443380000000000475,
1237662224092496776, 4429830000000001849, 3115540000000003218, 3101150000000002825,
3576680000000002728, 3605400000000000737, 3591040000000002325, 3298150000000002730,
3269340000000001270, 3312560000000004027, 3298160000000000456, 135916461], dtype=np.int)

In [ ]:
base1 = saga.object_catalog.load_single("Sopranos")

In [ ]:
find_near_objid(base, 3269460000000000929, 20)

In [ ]:
print_for_viewer(Query(C.is_low_z, QueryMaker.equals("TELNAME", "slack")).filter(base), ["OBJID","RA","DEC","ZQUALITY","SPEC_Z","MASKNAME","SPECOBJID"])

In [ ]:
lsb1 = read_marks_from_clipboard()

In [ ]:
t = Query(~C.is_sat, QueryMaker.in1d("OBJID", lsb1), "Mr_ < -12.3").filter(base)

plt.scatter(t["Mr_"], t["sb_r"])
t = Query(C.is_sat, QueryMaker.in1d("OBJID", lsb1), "Mr_ < -12.3").filter(base)

plt.scatter(t["Mr_"], t["sb_r"])

In [ ]:
t = Query(~C.is_sat, QueryMaker.in1d("OBJID", lsb1), "Mr_ < -12.3").filter(base)

plt.scatter(t["sb_r"], t["gr"])
t = Query(C.is_sat, QueryMaker.in1d("OBJID", lsb1), "Mr_ < -12.3").filter(base)

plt.scatter(t["sb_r"], t["gr"])

t = Query(~C.has_spec, "human_selected > 0", "Mr_ < -12.3").filter(base)

plt.scatter(t["sb_r"], t["gr"], marker="x", c="C5")

plt.xlim(22, 26)
plt.ylim(0.1, 0.8)

In [ ]:
t = Query(~C.is_sat, C.lsbg_cut, "Mr_ > -15.4", C.gr_cut,  C.has_spec, "Mr_ < -12.3").filter(base)

plt.scatter(t["sb_r"], t["gr"], s=3)
t = Query(C.is_sat, C.lsbg_cut, "Mr_ > -15.4", C.gr_cut, "Mr_ < -12.3").filter(base)

plt.scatter(t["sb_r"], t["gr"])

plt.xlim(22, 26)
plt.ylim(0.1, 0.8)

In [ ]:
Query(C.lsbg_cut, "Mr_ > -15.4", C.gr_cut, C.faint_end_limit, C.has_spec, "SPEC_Z < 0.1").filter(base, ["OBJID","RA","DEC"]).pformat_all()

In [ ]:
show_images(_, additional_keys=["SATS"])

In [ ]:
Query(~C.has_spec, C.lsbg_cut, C.gr_cut_tight, C.has_poor_spec, C.is_low_z).count(base)

In [ ]:
Query(C.has_spec, C.lsbg_cut, C.gr_cut_tight, "Mr_ >= -15.4", C.is_low_z).filter(base, ["OBJID", "RA", "DEC"]).pprint(-1, -1)

In [ ]:
Query(~C.has_spec, "human_selected", ~C.high_priority_gr, C.has_failed_spec).count(base)*0.1

In [ ]:
Query(~C.has_spec, "human_selected", C.has_poor_spec, C.is_sat).count(base)

In [ ]:
show_images(_)

In [ ]:
Query(~C.has_spec, C.has_poor_spec, "human_selected").count(base)

In [ ]:
Query(~C.has_spec, C.has_poor_spec, "human_selected", C.is_very_low_z).count(base)

In [ ]:
Query(C.lsbg_cut, C.gr_cut_tight, ~C.has_spec, C.has_been_targeted, "ZQUALITY == 2").count(base)

In [ ]:
Query("human_selected", C.has_been_targeted,  "ZQUALITY < 2").count(base)

In [ ]:
Query("human_selected", C.has_been_targeted, "ZQUALITY == 2", C.is_low_z).count(base)

In [ ]:
Query("log10(HOST_DIST * 1000 * radius / 3600 / 90 * arccos(0)) > -0.1 * (r_mag - log10(HOST_DIST) * 5 - 25 + 18)").count(base)

In [ ]:
Query("radius_ > 10**(-0.1*(Mr_+18))", "gr - abs(gr_err) < 0.75", ~C.has_spec, C.has_been_targeted, "ZQUALITY == 2", C.is_low_z).count(base)

In [ ]:
Query(C.high_priority_cuts, ~C.has_spec).count(base)

In [ ]:
plt.hist(Query(
    has_poor_z,
).filter(base,"SPEC_Z"), np.linspace(0, 1, 31), histtype="step", label="All low-ZQ objects");

plt.hist(Query(
    has_poor_z, "human_selected",
).filter(base,"SPEC_Z"), np.linspace(0, 1, 31), histtype="step", label="Human");

plt.xlabel("low-ZQ spec z")
plt.legend()

In [ ]:
t2 = Query("human_selected", C.is_low_z, has_poor_z, QueryMaker.contains("SPEC_REPEAT_ALL", "MMT")).filter(base, ["OBJID","RA_spec","DEC_spec", "SPEC_Z", "ZQUALITY", "MASKNAME", "SPECOBJID"])
print(len(t2))
t2["RA_spec"].format="%.6f"
t2["DEC_spec"].format="%.6f"
#t2.write("/home/yymao/Downloads/mmt_failed_specs.txt", format="ascii.basic")

In [ ]:
Query(C.basic_cut2, "sb_r >= 24.5", "0.4 <= gr", "gr <= 0.7", "r_mag >= 19", C.has_spec, C.has_our_specs_only).filter(base, ["OBJID", "RA", "DEC", "ZQUALITY"])

In [ ]:
Query(C.basic_cut2, "sb_r >= 24.5", "sb_r < 25", "0.5 <= gr", "gr <= 0.6", "r_mag >= 20.5", C.has_spec).filter(base)

## failure rate

In [ ]:
spec_by_aat_mmt = QueryMaker.contains("SPEC_REPEAT", "AAT") | QueryMaker.contains("SPEC_REPEAT", "MMT")
targeted_by_aat_mmt = QueryMaker.contains("SPEC_REPEAT_ALL", "AAT") | QueryMaker.contains("SPEC_REPEAT_ALL", "MMT")
failed_spec_aat_mmt = Query(targeted_by_aat_mmt, ~Query(spec_by_aat_mmt, C.has_spec))

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(11, 3.6))

for cuts, ax_this in zip(("(r_mag < 19)", "(19 <= r_mag) & (r_mag < 20)", "(20 <= r_mag) & (r_mag < 20.75)"), ax):    
#for cuts, ax_this in zip(("Mr < -16", "(-16 <= Mr) & (Mr < -13)", "(-13 <= Mr) & (Mr < -12.3)"), ax):    
    gal = Query(cuts, C.faint_end_limit, targeted_by_aat_mmt).filter(base)
    d = np.histogram2d((gal["gr"]), gal["sb_r"], bins=(np.linspace(0, 1, 11), np.linspace(21.5, 25.5, 9)))[0]
    gal1 = failed_spec_aat_mmt.filter(gal, ["gr", "sb_r"])
    d1 = np.histogram2d((gal1["gr"]), gal1["sb_r"], bins=(np.linspace(0, 1, 11), np.linspace(21.5, 25.5, 9)))[0]
    CS = ax_this.imshow((d1/d).T, origin="lower", extent=(0, 1, 21.5, 25.5), aspect="auto", cmap="plasma")

    ax_this.set_xlabel("Color ($g-r$)")
    #ax_this.set_title(cuts.replace("Mr) & (Mr", "Mr").replace("(", "").replace(")", ""))
    ax_this.set_title(cuts.replace("r_mag) & (r_mag", "r").replace("r_mag", "r").replace("(", "$").replace(")", "$").replace("<=", "\\leq"))
ax[0].set_ylabel("SB ($\\mu$)")
#fig.suptitle('Among objects that have NO redshifts', )
fig.tight_layout(pad=0.5)
plt.colorbar(CS, ax=ax, label="AAT/MMT Failure count", pad=0.02)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))

t = Query(C.basic_cut2, ~saga.host_catalog.construct_host_query("paper1_complete"), C.high_priority_cuts).filter(base)
w = Query(C.has_spec | targeted_by_aat_mmt).mask(t).astype(np.int)

d0 = np.histogram2d(t["r_mag"], t["sb_r"], bins=(np.linspace(17, 20.75, 11), np.linspace(21.5, 25.5, 9)))[0]
d1 = np.histogram2d(t["r_mag"], t["sb_r"], bins=(np.linspace(17, 20.75, 11), np.linspace(21.5, 25.5, 9)), weights=w)[0]
CS = ax[0].imshow((d1/d0).T, origin="lower", extent=(17, 20.75, 21.5, 25.5), aspect="auto", vmin=0, vmax=1)

d0 = np.histogram2d(t["r_mag"], t["gr"], bins=(np.linspace(17, 20.75, 11), np.linspace(0.05, 0.95, 10)))[0]
d1 = np.histogram2d(t["r_mag"], t["gr"], bins=(np.linspace(17, 20.75, 11), np.linspace(0.05, 0.95, 10)), weights=w)[0]
CS = ax[1].imshow((d1/d0).T, origin="lower", extent=(17, 20.75, 0.05, 0.95), aspect="auto", vmin=0, vmax=1)

r = np.linspace(17, 20.75, 10)
ax[0].plot(r, 0.6*(r-14) + 18.55, color='C3', ls="--", alpha=0.9)
#ax[0].plot(r, 0.7*(r-14) + 19.25, color='C3', ls="--", alpha=0.4)
ax[1].plot(r, -0.06*(r-14) + 0.9, color='C3', ls="--", alpha=0.9)
ax[1].plot(r, -0.06*(r-14) + 1.1, color='C3', ls="--", alpha=0.4)

ax[0].set_ylim(21.5, 25.5)
ax[1].set_ylim(0.05, 0.95)

ax[0].set_xlabel("$r$")
ax[0].set_ylabel("SB ($\\mu$)")
ax[1].set_xlabel("$r$")
ax[1].set_ylabel("Color ($g-r$)")

fig.suptitle('Targeting rate (Paper2 "Newly Complete" hosts; within Paper2 targeting cuts)')
fig.tight_layout(pad=0.8)
plt.colorbar(CS, ax=ax, label="Targeting Rate", pad=0.02)

## Quenched LSB

In [ ]:
Query(C.basic_cut2, "sb_r >= 24.5", "0.4 <= gr", "gr <= 0.8", "-15 <= Mr_", "Mr_ < -12.295", C.has_spec, C.has_aat_mmt_spec, C.has_our_specs_only).filter(base, ["OBJID", "RA", "DEC"]).pprint(-1, -1)

In [ ]:
Query("human_selected", QueryMaker.equals("HOSTID", "nsa165980"), C.high_priority_cuts).filter(base)

In [ ]:
find_near_ra_dec(saga.host_catalog.load(), 228, 60, 3600)

In [ ]:
plt.hist(np.log10(Query("human_selected").filter(base, "p_sat_approx")), 15);
plt.xlabel(r'$\log(\mathcal{R}_\mathrm{sat})$')

In [ ]:
t2 = Query("human_selected").filter(base, ["OBJID", "RA", "DEC", "HOSTID", "SPEC_Z", "ZQUALITY", "TELNAME", "MASKNAME", "SPECOBJID"])
t2["SPEC_Z"] = np.nan_to_num(t2["SPEC_Z"], nan=-1)
t2["RA"].format="%.8f"
t2["DEC"].format="%.8f"
t2.write("/home/yymao/Downloads/human_selected.csv", overwrite=True)